In [38]:
from CompactFIPS202 import *
import binascii
import os
from hashlib import sha3_256, sha3_512, shake_128, shake_256
from kyber import Kyber512, Kyber768, Kyber1024
from aes256_ctr_drbg import AES256_CTR_DRBG

def parse_kat_data(data):
    parsed_data = {}
    count_blocks = data.split('\n\n')
    for block in count_blocks[1:-1]:
        block_data = block.split('\n')
        count, seed, pk, sk, ct, ss = [line.split(" = ")[-1] for line in block_data]
        parsed_data[count] = {
            "seed": bytes.fromhex(seed),
            "pk": bytes.fromhex(pk),
            "sk": bytes.fromhex(sk),
            "ct": bytes.fromhex(ct),
            "ss": bytes.fromhex(ss),   
        }
    return parsed_data
    


In [39]:
with open("assets/PQCkemKAT_1632.rsp") as f:
    kat_data = f.read()
    parsed_data = parse_kat_data(kat_data)

data = parsed_data['0']
seed, pk, sk, ct, ss = data.values()
        
# Seed DRBG with KAT seed
Kyber512.set_drbg_seed(seed)
# Assert keygen matches
_pk, _sk = Kyber512.keygen()

# Assert encapsulation matches
_ct, _ss = Kyber512.enc(_pk)

#    # Assert decapsulation matches
#    __ss = Kyber512.dec(ct, sk)

In [40]:
seed.hex()

'061550234d158c5ec95595fe04ef7a25767f2e24cc2bc479d09d86dc9abcfde7056a8c266f9ef97ed08541dbd2e1ffa1'

In [41]:
pk == _pk

True

In [42]:
(pk == _pk) == (sk == _sk) == (ct == _ct) == (ss == _ss)

True

In [43]:
random_bytes = os.urandom

In [44]:
a = random_bytes(32)

In [45]:
len(Kyber512._xof_a)

1

In [46]:
len(Kyber512._xof_input_bytes)

34

In [47]:
Kyber512._xof_a.hex()

'01'

In [48]:
a = shake_128(Kyber512._xof_input_bytes).digest(Kyber512._xof_length).hex()

In [49]:
b = SHAKE128(Kyber512._xof_input_bytes, Kyber512._xof_length).hex()

In [50]:
a == b

True

In [51]:
a

'85d62974496e87c185a83a7b25587a85fd15a099bf294b5a5f8cf0d44749e292f598c3f2b036aad7057866341b19c98cc2f5f29cf8faab2ab005c421838e3af73de6f2f3af1bb5cd1337fbb339e9e99734062326226b9e27533995f690092bc65bece926bc7376bb2c82931a530c3d89b595bf3aefb95e4aa8b4e966a4497a4ccda4cf11c9b634d2eff83c20664a9e04347039fa9271f856f58673be6cb53d035ca8ae368fe6cb9367e2ab8cf1787b884c2805830ab35c4c5cb83440dde77f18675b70e59d8e3b4c2192d636e3e80d0464da8e0b94c70291590cccbd202fac317d847636cefc065547800949a2b426352ab4e3ce5f879fcfe0cdc5e7beb0aabe769e24e18ac02b55ffdcfb28aa5093665d0cb2ff378659b72b8e23f03a5e87a02f027ce8f32568635cf6ce8c79cef2efa50fd67eeb64789ad6f302eeefd16a15fad69b691beb6b4eb4e2c7546ce46c7c904cf17df8c0cd57e99c32579adb4ebf710a9d7c2468127003f49cbb6c84b7d8ba210f66e9297d156faf7abec5f79e3811f463040c14b79549489a4fb50b8127268b31215f7a3d0ce36993b7e3cbf563839f4dfd39b7fa85834e90a3fe17d446f957392dd73ffcbdd45ea65dd458715476f93da2f8022aaf524d210d4dc8cd14150d58bdcd02e8514794a286a8323e5ed2dcf0b28a7054745520b51cf559c71655d0683

In [52]:
a = sha3_256(Kyber512._xof_input_bytes).digest().hex()

In [53]:
b = SHA3_256(Kyber512._xof_input_bytes).hex()

In [54]:
a == b

True

In [55]:
a

'20658677b2e1f5f524759dc04a97f238856e5031e972cd9e52387e147e325bab'

In [56]:
b

'20658677b2e1f5f524759dc04a97f238856e5031e972cd9e52387e147e325bab'

In [60]:
state = bytearray([0 for i in range(2)])
state

bytearray(b'\x00\x00')

In [66]:
1089//8

136